# Overall Design philosophy: type tree and multiple dispatch

In this IJulia notebook we will discuss the overall design of the Julia language. Before we start, however, I should tell you how I made this notebook so you can use them for your homworks and projects (if you want).

## Quick Julia Tip
To use IJulia notebooks you will first need the Jupyter notebooks working on python (they come pre-installed with Anaconda). Now you can download the IJulia package at the Julia command line with:

```julia
julia> Pkg.add("IJulia")
```
Now load `IJulia` into the namespace with.
```julia
julia> using IJulia
```
To start a notebook (which will open your default web browser) you can do

```julia
julia> notebook()
```
or 
```julia
julia> notebook(detached=true)
```
which will run the notebook in the background and give back the Julia prompt.

Another useful package is `NBInclude` which allows you to run a whole Julia notebook, at the command line, and import all the defined variables into the command line namespace.

## Types in Julia

Every variable in Julia has a concrete type

In [1]:
typeof(4)

Int64

In [2]:
typeof(4.0)

Float64

In [3]:
typeof(4//7)

Rational{Int64}

In [4]:
typeof(rand(101,10))

Array{Float64,2}

In [5]:
typeof([1, 2, 3]) # 1-d array of ints

Array{Int64,1}

Function also have a type

In [6]:
typeof(rand)

Function

Types have a type

In [7]:
typeof(Float64) # convention, types start with capital letter

DataType

In [8]:
typeof(typeof(Float64))

DataType

### Typing is not static

In [9]:
a = 4

4

In [10]:
typeof(a)

Int64

In [11]:
a = 1.0

1.0

In [12]:
typeof(a)

Float64

### Besides concrete types, there are abstract types

* Concrete types correspond to a definitive layout in memory (Int16, Int64, Array{Bool, 2})
* Abstract types correspond to a collection or catigory of concrete types.
* For example both Int64 and Float64 are a subtype of Real which is a subtype of Number, etc...
* This gives a whole tree of types.
* Leaf nodes of the tree are concrete types (these types have a specific way they are layed out in memory)
* Parent nodes are abstract types (which don't have a fixed memory layout configuration)
* For example Int16 and Int64 are both of abstract type Real but have different memory layouts

You can move up tree with super

In [13]:
super(Int64)

Signed

In [14]:
super(Int64) |> super

Integer

In [15]:
super(Int64) |> super |> super

Real

In [16]:
super(Int64) |> super |> super |> super

Number

At the top of the tree we have Any

In [17]:
super(Int64) |> super |> super |> super |> super

Any

You can move down the tree with subtypes

In [18]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16 
 Float32 
 Float64 

In [19]:
subtypes(AbstractArray)

18-element Array{Any,1}:
 AbstractSparseArray{Tv,Ti,N}                                                                  
 Base.LinAlg.AbstractTriangular{T,S<:AbstractArray{T,2}}                                       
 Base.LinAlg.HessenbergQ{T,S<:AbstractArray{T,2}}                                              
 Base.LinAlg.QRCompactWYQ{S,M<:AbstractArray{T,2}}                                             
 Base.LinAlg.QRPackedQ{T,S<:AbstractArray{T,2}}                                                
 Base.LinAlg.SVDOperator{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S}        
 Base.SparseMatrix.CHOLMOD.FactorComponent{Tv,S}                                               
 Bidiagonal{T}                                                                                 
 Core.Inference.Range{T}                                                                       
 DenseArray{T,N}                                                                               
 Diagonal{T}   

Check if a type is an ancestor 

In [20]:
AbstractFloat <: Number

true

In [21]:
AbstractFloat <: AbstractArray

false

* You can make user defined types easy too (we'll see that in a second).
* The type system is a bit more exposed in Julia than in matlab
* The reason is multiple dispatch.
* Instead of having the OO style where functions "live" with the objection or types, types and functions are seperated
* To specify the behavor of a function on a new type or object, one can annotate the types of the arguments

I'm defining 4 different versions of foo.
 

In [22]:
function foo(x::AbstractFloat, y::AbstractFloat)
        println("foo(Float, Float) was called")
        round(Int, x * y)
end

function foo(x::Integer, y::Integer)
        println("foo(Int, Int)  was called")
        x * y
end

function foo(x, y) # fall back
        println("fall back was called")
        round(Int, x .* y)
end

function foo(x)
        foo(x, x)
end

foo (generic function with 4 methods)

In [23]:
foo(1, 1)

foo(Int, Int)  was called


1

In [24]:
foo(2.0, 1)

fall back was called


2

In [25]:
foo(2.0, 4.9)

foo(Float, Float) was called


10

In [26]:
foo(2.0)

foo(Float, Float) was called


4

In [27]:
foo(randn(15, 15))

fall back was called


15x15 Array{Int64,2}:
 1  0  2  0  0  1  0  0  0  4  1  3  0  1  2
 0  0  1  0  1  0  0  0  0  1  0  0  1  0  6
 5  0  1  0  1  0  0  0  1  1  0  1  1  0  0
 0  0  1  1  0  1  0  0  0  1  0  5  1  0  0
 2  0  3  0  0  1  0  0  0  0  1  1  4  1  0
 2  2  2  0  0  0  1  0  0  1  0  2  0  2  0
 4  1  0  2  2  2  0  1  2  1  0  1  0  2  0
 0  0  0  0  0  0  0  1  0  1  0  0  0  1  2
 0  0  1  4  0  0  0  1  0  4  2  0  2  0  2
 2  2  1  1  0  0  0  0  0  0  1  0  0  2  1
 0  0  0  0  0  0  3  0  0  0  0  0  0  0  1
 4  5  5  0  1  0  0  0  2  0  2  1  0  1  0
 1  0  2  0  0  2  0  1  2  2  1  1  1  0  1
 0  3  1  0  0  0  0  0  0  0  4  0  0  0  1
 0  1  0  0  1  0  0  1  0  1  1  1  0  0  1

Lists all the possible call signatures
 

In [28]:
methods(foo)

# 4 methods for generic function "foo":
foo(x::AbstractFloat, y::AbstractFloat) at In[22]:2
foo(x::Integer, y::Integer) at In[22]:7
foo(x) at In[22]:17
foo(x, y) at In[22]:12

### Just-in-time compilation (JIT)

* When `foo(1, 1)` is called it looks up the type signature of the arguments and decides which function to use
* It also, checks to see if you have called that function with those same type arguments,
* If not it tries to JIT complile it and will save that compiled version for the next time you use it with those same type of arguments

In [29]:
a = 2

2

Warm up Jit compile

In [30]:
@time foo(a)

foo(Int, Int)  was called
  

4

0.001345 seconds (684 allocations: 78.397 KB)


Now we are using the Jit

In [31]:
@time foo(a)

foo(Int, Int)  was called


4

  0.000194 seconds (309 allocations: 14.422 KB)


* So it is very easy to optimize some functions for different types of arguments.
* but you need to write your code so Julia can infer the types to compile it
* Note: Defining with type anotation like `foo(x::Float64, y::Int)` will be just as fast as `foo(x, y)`. In particular, when foo(Float64, Int) is called, it already knows the types of the arguments and will compile for that type signature.

### Type stability
When functions are not type stable, the JIT doesn't work (as effectively)

The following function is not type stable.
 

In [32]:
function baz(x)
    cntr = 0        # starts as as int
    for i = 1:length(x)
        if x[i] > 0
            cntr += 1.0 # depending on the run time values  might promote to a float
        end
    end
    cntr
end

baz (generic function with 1 method)

This one is type stable
 

In [33]:
function boo(x)
    cntr = 0.0        # starts as as float
    for i = 1:length(x)
        if x[i] > 0
        	cntr += 1.0 # stays a float
        end
    end
    cntr
end

boo (generic function with 1 method)

In [34]:
a = rand(1_000_000)

1000000-element Array{Float64,1}:
 0.689822 
 0.723767 
 0.329307 
 0.350774 
 0.494701 
 0.0848251
 0.0556951
 0.264839 
 0.94545  
 0.443409 
 0.518686 
 0.208869 
 0.28026  
 ⋮        
 0.77693  
 0.594362 
 0.347436 
 0.983789 
 0.83868  
 0.62381  
 0.864981 
 0.221135 
 0.792031 
 0.81677  
 0.81722  
 0.0414312

In [35]:
@time baz(a)

1.0e6

0.034873 seconds (1.00 M allocations: 15.395 MB, 10.58% gc time)


In [36]:
@time baz(a)

1.0e6

0.027558 seconds (1.00 M allocations: 15.259 MB)


In [37]:
@time boo(a)

1.0e6

0.007345 seconds (2.24 k allocations: 111.458 KB, 30.49% gc time)


In [38]:
@time boo(a)

1.0e6

0.001363 seconds (5 allocations: 176 bytes)


### We can look under the hood at the type inference

In [39]:
@code_warntype baz(a)

Variables:
  x::Array{Float64,1}
  cntr::ANY
  #s41::Int64
  i::Int64
  ####fx#1710#8037::Float64

Body:
  begin  # In[32], line 2:
      cntr = 0 # In[32], line 3:
      GenSym(2) = (Base.arraylen)(x::Array{Float64,1})::Int64
      GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,GenSym(2))::Bool,GenSym(2),(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64)))
      #s41 = (top(getfield))(GenSym(0),:start)::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Bool)) goto 1
      2: 
      GenSym(4) = #s41::Int64
      GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s41::Int64,1))
      i = GenSym(4)
      #s41 = GenSym(5) # In[32], line 4:
      GenSym(3) = (Base.arrayref)(x::Array{Float64,1},i::Int64)::Float64
      ####fx#1710#8037 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
      unless (Base.box)(Base.Bool,(Base.or_int)(

In [40]:
@code_warntype boo(a)

Variables:
  x::Array{Float64,1}
  cntr::Float64
  #s41::Int64
  i::Int64
  ####fx#1710#8038::Float64

Body:
  begin  # In[33], line 2:
      cntr = 0.0 # In[33], line 3:
      GenSym(2) = (Base.arraylen)(x::Array{Float64,1})::Int64
      GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,GenSym(2))::Bool,GenSym(2),(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64)))
      #s41 = (top(getfield))(GenSym(0),:start)::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Bool)) goto 1
      2: 
      GenSym(4) = #s41::Int64
      GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s41::Int64,1))
      i = GenSym(4)
      #s41 = GenSym(5) # In[33], line 4:
      GenSym(3) = (Base.arrayref)(x::Array{Float64,1},i::Int64)::Float64
      ####fx#1710#8038 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
      unless (Base.box)(Base.Bool,(Base.or

### We can really look under the hood at the lowered code

In [41]:
@code_lowered baz(a)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:x], Any[Any[Any[:x,:Any,0],Any[:cntr,:Any,2],Any[symbol("#s41"),:Any,2],Any[:i,:Any,18]],Any[],2,Any[]], :(begin  # In[32], line 2:
        cntr = 0 # In[32], line 3:
        GenSym(0) = (Main.colon)(1,(Main.length)(x))
        #s41 = (top(start))(GenSym(0))
        unless (top(!))((top(done))(GenSym(0),#s41)) goto 1
        2: 
        GenSym(1) = (top(next))(GenSym(0),#s41)
        i = (top(getfield))(GenSym(1),1)
        #s41 = (top(getfield))(GenSym(1),2) # In[32], line 4:
        unless (Main.getindex)(x,i) > 0 goto 4 # In[32], line 5:
        cntr = cntr + 1.0
        4: 
        3: 
        unless (top(!))((top(!))((top(done))(GenSym(0),#s41))) goto 2
        1: 
        0:  # In[32], line 8:
        return cntr
    end))))

In [42]:
@code_lowered boo(a)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:x], Any[Any[Any[:x,:Any,0],Any[:cntr,:Any,2],Any[symbol("#s41"),:Any,2],Any[:i,:Any,18]],Any[],2,Any[]], :(begin  # In[33], line 2:
        cntr = 0.0 # In[33], line 3:
        GenSym(0) = (Main.colon)(1,(Main.length)(x))
        #s41 = (top(start))(GenSym(0))
        unless (top(!))((top(done))(GenSym(0),#s41)) goto 1
        2: 
        GenSym(1) = (top(next))(GenSym(0),#s41)
        i = (top(getfield))(GenSym(1),1)
        #s41 = (top(getfield))(GenSym(1),2) # In[33], line 4:
        unless (Main.getindex)(x,i) > 0 goto 4 # In[33], line 5:
        cntr = cntr + 1.0
        4: 
        3: 
        unless (top(!))((top(!))((top(done))(GenSym(0),#s41))) goto 2
        1: 
        0:  # In[33], line 8:
        return cntr
    end))))

### We can really look under the hood at the llvm code

In [43]:
@code_llvm baz(a)


define %jl_value_t* @julia_baz_22310(%jl_value_t*, %jl_value_t**, i32) {
top:
  %3 = alloca [5 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [5 x %jl_value_t*]* %3, i64 0, i64 0
  %4 = getelementptr [5 x %jl_value_t*]* %3, i64 0, i64 2
  %5 = getelementptr [5 x %jl_value_t*]* %3, i64 0, i64 3
  store %jl_value_t* inttoptr (i64 6 to %jl_value_t*), %jl_value_t** %.sub, align 8
  %6 = load %jl_value_t*** @jl_pgcstack, align 8
  %7 = getelementptr [5 x %jl_value_t*]* %3, i64 0, i64 1
  %.c = bitcast %jl_value_t** %6 to %jl_value_t*
  store %jl_value_t* %.c, %jl_value_t** %7, align 8
  store %jl_value_t** %.sub, %jl_value_t*** @jl_pgcstack, align 8
  store %jl_value_t* null, %jl_value_t** %4, align 8
  store %jl_value_t* null, %jl_value_t** %5, align 8
  %8 = getelementptr [5 x %jl_value_t*]* %3, i64 0, i64 4
  store %jl_value_t* null, %jl_value_t** %8, align 8
  %9 = load %jl_value_t** %1, align 8
  store %jl_value_t* inttoptr (i64 4434321488 to %jl_value_t*), %jl_value_t** %4

In [44]:
@code_llvm boo(a)


define double @julia_boo_22311(%jl_value_t*) {
top:
  %1 = getelementptr inbounds %jl_value_t* %0, i64 1
  %2 = bitcast %jl_value_t* %1 to i64*
  %3 = load i64* %2, align 8
  %4 = icmp sgt i64 %3, 0
  %5 = select i1 %4, i64 %3, i64 0
  %6 = icmp eq i64 %5, 0
  br i1 %6, label %L4, label %L.preheader

L.preheader:                                      ; preds = %top
  %7 = load i64* %2, align 8
  %8 = bitcast %jl_value_t* %0 to i8**
  br label %L

L:                                                ; preds = %L2, %L.preheader
  %"#s41.0" = phi i64 [ %18, %L2 ], [ 1, %L.preheader ]
  %cntr.0 = phi double [ %cntr.1, %L2 ], [ 0.000000e+00, %L.preheader ]
  %9 = add i64 %"#s41.0", -1
  %10 = icmp ult i64 %9, %7
  br i1 %10, label %idxend, label %oob

oob:                                              ; preds = %L
  %11 = alloca i64, align 8
  store i64 %"#s41.0", i64* %11, align 8
  call void @jl_bounds_error_ints(%jl_value_t* %0, i64* %11, i64 1)
  unreachable

idxend:                        

### We can really look under the hood at the machine code

In [45]:
@code_native baz(a)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[32]
Source line: 2
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$40, %rsp
	movq	$6, -80(%rbp)
	movabsq	$jl_pgcstack, %rcx
	movq	(%rcx), %rax
	movq	%rax, -72(%rbp)
	leaq	-80(%rbp), %rax
	movq	%rax, (%rcx)
	vxorpd	%xmm0, %xmm0, %xmm0
	vmovupd	%xmm0, -64(%rbp)
	movq	$0, -48(%rbp)
	movq	(%rsi), %r12
	movabsq	$4434321488, %rax       ## imm = 0x1084E6050
	xorl	%ebx, %ebx
Source line: 2
	movq	%rax, -64(%rbp)
Source line: 3
	movq	8(%r12), %r13
	testq	%r13, %r13
	movl	$0, %ecx
	cmovnsq	%r13, %rcx
	testq	%rcx, %rcx
	je	L247
Source line: 5
	testq	%r13, %r13
	cmovsq	%rbx, %r13
Source line: 3
	negq	%r13
	movabsq	$4434321488, %rax       ## imm = 0x1084E6050
Source line: 5
	movabsq	$jl_apply_generic, %r15
	xorl	%r14d, %r14d
Source line: 4
L144:	cmpq	8(%r12), %rbx
	jae	L279
Source line: 5
	leaq	(,%r14,8), %rcx
Source line: 4
	movq	(%r12), %rdx
Source line: 5
	subq	%rcx, %rdx


In [46]:
@code_native boo(a)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[33]
Source line: 3
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 3
	movq	8(%rdi), %r9
	xorl	%ecx, %ecx
	testq	%r9, %r9
	movl	$0, %edx
	cmovnsq	%r9, %rdx
	vxorps	%xmm0, %xmm0, %xmm0
	testq	%rdx, %rdx
	je	L129
Source line: 5
	testq	%r9, %r9
	cmovsq	%rcx, %r9
Source line: 3
	negq	%r9
Source line: 4
	movq	8(%rdi), %r8
	vxorpd	%xmm1, %xmm1, %xmm1
	movabsq	$13272309728, %rax      ## imm = 0x317175FE0
	vmovsd	(%rax), %xmm2
	xorl	%esi, %esi
	vxorps	%xmm0, %xmm0, %xmm0
L73:	cmpq	%r8, %rcx
	jae	L134
Source line: 5
	leaq	(,%rsi,8), %rdx
Source line: 4
	movq	(%rdi), %rax
Source line: 5
	subq	%rdx, %rax
Source line: 4
	vmovsd	(%rax), %xmm3
	vucomisd	%xmm1, %xmm3
	jbe	L114
Source line: 5
	vaddsd	%xmm2, %xmm0, %xmm0
L114:	incq	%rcx
	decq	%rsi
	cmpq	%rsi, %r9
	jne	L73
Source line: 8
L129:	movq	%rbp, %rsp
	popq	%rbp
	ret
L134:	movl	$1, %eax
Source line: 4
	subq	%rsi, %rax
	movq	%rsp, %rcx
	leaq	-16(%rcx), %rsi
	movq	%rsi, %rsp
	movq	%rax, -16(%

### Julia has been designed, from the groud up, to make type inference easier

Notice that `sqrt(-1.0)` gives an error but `sqrt(-1.0+0.0im)` works. 

In [47]:
sqrt(-1.0) #<--- error

LoadError: LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).
while loading In[47], in expression starting on line 1

Why? The problem is that `sqrt(x)` may return either a float or a complex float depending on the run time values of `x` and not the type of `x`. Therefore, Julia throws an error in the case `x<0` so the type inference can always guarentee that `sqrt(Float)` will be a float.

In [48]:
sqrt(-1.0+0im) # <---- works

0.0 + 1.0im

### Method dispatch in action

* A great example is the Distributions package

In [49]:
x = rand(10)

10-element Array{Float64,1}:
 0.782917 
 0.0470413
 0.801526 
 0.728042 
 0.752398 
 0.915504 
 0.640927 
 0.328117 
 0.127831 
 0.348905 

In [50]:
mean(x), std(x)  # functions in Base Julia

(0.5473208122889365,0.30783004827364197)

In [51]:
 using Distributions
λ, α, β = 5.5, 0.1, 0.9
xrv = Beta(α, β) # creats an instance of a Beta random variable
yrv = Poisson(λ) # creats  an instance of a Poisson
zrv = Poisson(λ) # another instance

Distributions.Poisson(λ=5.5)

In [52]:
typeof(xrv), typeof(yrv), typeof(zrv)

(Distributions.Beta,Distributions.Poisson,Distributions.Poisson)

`mean` is overloaded to give the random variable expected value.

In [53]:
 mean(xrv)  # expected value of a Beta(0.1, 0.9)

0.1

`std` is overloaded to give the random variable standard deviation

In [54]:
std(zrv)   # std of a Poisson(5.5)

2.345207879911715

`rand` is overloaded to give random samples from yrv

In [55]:
rand(yrv, 10)  # Poisson(5.5) samples

10-element Array{Int64,1}:
 7
 3
 5
 7
 5
 9
 3
 6
 6
 4

Check which method is called

In [56]:
@which mean(xrv)

mean(d::Distributions.Beta) at /Users/ethananderes/.julia/v0.4/Distributions/src/univariate/continuous/beta.jl:22

This will allow you to see the Julia source which was called.
```julia
julia> @edit mean(xrv)
```

If you have Julia source you can go directly to code.
This is particularly useful when debugging (you can read the source to see what is going wrong).

In [57]:
mean(["hi"])

LoadError: LoadError: MethodError: `/` has no method matching /(::ASCIIString, ::Int64)
Closest candidates are:
  /(!Matched::Integer, ::Integer)
  /(!Matched::Complex{T<:Real}, ::Real)
  /(!Matched::BigFloat, ::Union{Int16,Int32,Int64,Int8})
  ...
while loading In[57], in expression starting on line 1

Lets see where the definition of mean to see what is going wrong

In [58]:
@which mean(["hi"])

mean(A::AbstractArray{T,N}) at statistics.jl:19

# Writing fast code in Julia

* Write most of you code in functions
* Avoid globals (if you must use them, declare as `const` which can not change types)
* Write type stable functions
* Pre allocate arrays and use `for` loops
* Optimize using the profiler


## User defined types and metaprograming

Here is how a user can defined a new type

In [59]:
type Hmatrix
        frstcol :: Vector{Float64} # field names and optional type annotation
        lastcol :: Vector{Float64} # this is where type annotation is important for speed, want these to be concrete types
end

Creat an instance of two Hmatrices. Basic default constructor

In [60]:
anHmat = Hmatrix([1.0 ,2.0 ,3.0], [5.0 ,6.0, 7.0])
bnHmat = Hmatrix([0.0 ,2.0 ,3.0], [0.0 ,6.0, 7.0])

Hmatrix([0.0,2.0,3.0],[0.0,6.0,7.0])

Getting the field values.

In [61]:
anHmat.frstcol
anHmat.lastcol

3-element Array{Float64,1}:
 5.0
 6.0
 7.0

Use dispatch to generate other constructors

In [62]:
Hmatrix(x::Vector{Float64}) = Hmatrix(x, copy(x))
Hmatrix(x::Float64, d::Int64) = Hmatrix(fill(x,d))
Heye(d::Int64) = Hmatrix(fill(1.0,d))

Heye (generic function with 1 method)

Now I can define operations on Hmatrix.

In [63]:
import Base: sin, size
sin(h::Hmatrix)  = Hmatrix(sin(h.frstcol), sin(h.lastcol))
size(h::Hmatrix) = (length(h.frstcol), length(h.frstcol))
sin(anHmat)

Hmatrix([0.8414709848078965,0.9092974268256817,0.1411200080598672],[-0.9589242746631385,-0.27941549819892586,0.6569865987187891])

In [64]:
size(anHmat)

(3,3)

This gets tedious after a while so I program over the language itself: metaprograming

In [65]:
import Base: exp, abs, tan, log
for op in [:exp, :abs, :tan, :log]
        quote
                $op(h::Hmatrix) = Hmatrix($op(h.frstcol), $op(h.lastcol))
        end |> eval
end

In [66]:
abs(anHmat)

Hmatrix([1.0,2.0,3.0],[5.0,6.0,7.0])

In [67]:
tan(anHmat)

Hmatrix([1.5574077246549023,-2.185039863261519,-0.1425465430742778],[-3.380515006246586,-0.29100619138474915,0.8714479827243187])

Pairwise operations with Hmatrices.

In [68]:
import Base: .+, .-, .*, ./
for op in [:.+, :.-, :.*, :./]
        quote
                function $op(h::Hmatrix, g::Hmatrix)
                        frstcol = $op(h.frstcol, g.frstcol)
                        lastcol = $op(h.lastcol, g.lastcol)
                        return Hmatrix(frstcol, lastcol)
                end
        end |> eval
end
import Base: *
function *(mat::Matrix, hmat::Hmatrix)
        Hmatrix(mat * hmat.frstcol, mat * hmat.lastcol)
end

* (generic function with 149 methods)

A pre-existing function on matrices which only used the above operations will automatically work on Hmatrices.
I encountered this when a new type I had made, just worked on some Runge-Kutta code

In [69]:
function expmm(mat)
        mat .+ rand(size(mat)) * mat .+ (rand(size(mat)).^2) * mat
end
expmm(eye(2))

2x2 Array{Float64,2}:
 1.59398   0.533019
 0.957554  2.51565 

In [70]:
expmm(anHmat)

Hmatrix([8.173963163731788,7.865702167344595,8.729008690865706],[24.43114386123764,25.35935879812256,23.7770484574838])

I can make indexing work like regular arrays

In [71]:
import Base: getindex
function getindex(H::Hmatrix, i::Integer, j::Integer)
        siz, _ = size(H)
        if j == 1
                return H.frstcol[i]
        elseif j == siz
                return H.lastcol[i]
        else
                return 0.0
        end
end

getindex (generic function with 138 methods)

In [72]:
anHmat

Hmatrix([1.0,2.0,3.0],[5.0,6.0,7.0])

In [73]:
anHmat[2,3]

6.0

### Putting a custom type in the type tree and inhert behavior

Take a look at the source code

In [74]:
@which Symmetric(rand(3,3))

call(::Type{Symmetric{T,S<:AbstractArray{T,2}}}, A::AbstractArray{T,2}) at linalg/symmetric.jl:8

In [75]:
immutable H2matrix{T} <: AbstractMatrix{T}
        frstcol ::Vector{T}
        lastcol ::Vector{T}
end

Defines a whole class of types: `H2matrix{Float64}`, `H2matrix{Int64}`, etc
These are ancestors of `AbstractMatrix{T}`
The fact that they are ancestors of AbstractArrays means that other function will just work.
Moreover you can specialize fast methods if your working with `H2matrix{Uint16}`, for example

I can make indexing work like regular arrays

In [76]:
import Base: getindex, size, length, sum
size(h::H2matrix) = (length(h.frstcol), length(h.lastcol))
length(h::H2matrix) = length(h.frstcol)^2
function getindex{T}(H::H2matrix{T}, i::Integer, j::Integer)
        siz, _ = size(H)
        if j == 1
            return H.frstcol[i]
        elseif j == siz
            return H.lastcol[i]
        else
            return convert(T, 0)
        end
end

getindex (generic function with 139 methods)

In the above example, this is an easy way to specialize code based on the element type of H2matrix

In [77]:
sum(h::H2matrix) = sum(h.frstcol) + sum(h.lastcol)

sum (generic function with 18 methods)

Now stuff "Just Works" due to an ancestors of AbstractMatrix{T}

In [78]:
anH2 = H2matrix(rand(5), rand(5)) # printing is inherited from AbstractMatrix{T}

5x5 H2matrix{Float64}:
 0.302959  0.0  0.0  0.0  0.107827
 0.577481  0.0  0.0  0.0  0.740502
 0.157785  0.0  0.0  0.0  0.457961
 0.557784  0.0  0.0  0.0  0.856688
 0.782605  0.0  0.0  0.0  0.286424

In [79]:
anH2 = H2matrix([1,2,3], [3,2,1]) # spcialization for integer

3x3 H2matrix{Int64}:
 1  0  3
 2  0  2
 3  0  1

In [80]:
mean(anH2)

1.3333333333333333

How did this work? Check out the source.

```julia
@edit mean(anH2)
```

There is a fallback method for `mean(A::AbstractArray)`` which only uses sum and length.
You can see how important it is to be able read look at Julia source
This wouldn't be possible if it wasn't both fast and readable at the same time